# Homework 4 - Principal Component Analysis
## Data Analysis
### FINM August Review 

Mark Hendricks

hendricks@uchicago.edu

## 1. Principal Components

Use the single-name equity data from the data set mentioned above. (That is, leave out SPY and SHV.)

In [46]:
# Import packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression

In [36]:
# Import data 
returns = pd.DataFrame(pd.read_excel(r'../data/spx_weekly_returns_single_names.xlsx', sheet_name= 's&p500 rets')).set_index('date')
spy = pd.DataFrame(pd.read_excel(r'../data/spx_weekly_returns_single_names.xlsx', sheet_name= 'benchmark rets')).set_index('date')[['SPY']]

1. Calculate the principal components of the return series.

In [37]:
# Calculate principal components of the return series
pca = PCA(svd_solver='full')
pca.fit(returns.values)

pca_factors = pd.DataFrame(
    pca.transform(returns.values),
    columns = ['PC {}'.format(i+1) for i in range(pca.n_components_)],
    index= pd.to_datetime(returns.index)
)

display(pca_factors.tail().style.format('{:,.4f}'))


2. Report the eigenvalues associated with these principal components. Report each eigenvalue as a percentage of the sum of all the eigenvalues. This is the total variation each PCA explains.


In [38]:
explained_variance = pd.DataFrame(
    pca.explained_variance_ratio_,
    index=pca_factors.columns,
    columns= ['Explained Variance']
)

explained_variance['Cumulative Explained Variance'] = explained_variance['Explained Variance'].cumsum()

display(explained_variance.head(20).style.format('{:,.2%}'))

,Explained Variance,Cumulative Explained Variance
PC 1,38.91%,38.91%
PC 2,5.67%,44.58%
PC 3,3.78%,48.36%
PC 4,2.77%,51.14%
PC 5,2.07%,53.21%
PC 6,1.49%,54.70%
PC 7,1.25%,55.95%
PC 8,1.16%,57.11%
PC 9,1.05%,58.16%
PC 10,1.03%,59.19%


3. How many PCs are needed to explain 75% of the variation?
A: We need 40 principal components

In [39]:
explained_variance[explained_variance['Cumulative Explained Variance'] >= 0.75].index[0]

'PC 40'

4. Calculate the correlation between the first (largest eigenvalue) principal component with each of the 22 single-name equities. Which correlation is highest?

PRU with 86%.

In [40]:
returns_corr = returns.copy()
returns_corr.insert(0, 'PC 1', pca_factors['PC 1'])

corr_table = pd.DataFrame(returns_corr.corr().iloc[0, 1:]).sort_values('PC 1', ascending= False).style.format('{:,.2%}')
display(corr_table)

,PC 1
PRU,86.18%
PFG,85.50%
TEL,84.56%
PH,83.96%
AMP,83.69%
DOV,82.54%
HON,82.37%
MET,82.33%
EMN,81.93%
APH,81.84%



5. Calculate the correlation between the SPY and the first, second, third principal components.

In [41]:
factors = pca_factors[['PC 1', 'PC 2', 'PC 3']]
factors['SPY'] = spy
print('Correlation with SPY:')
display(factors.corr().iloc[3, :3])

Correlation with SPY:


/var/folders/8m/h9p4w9xx385dnwz_x2453dyc0000gn/T/ipykernel_44134/1521310091.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factors['SPY'] = spy


PC 1    0.941321
PC 2   -0.223747
PC 3    0.060345
Name: SPY, dtype: float64

***

## 2. PCR and PLS

1. Principal Component Regression (PCR) refers to using PCA for dimension reduction, and then utilizing the principal components in a regression. Try this by regressing SPY on the first 3 PCs calculated in the previous section. Report the r-squared.


In [44]:
pcr = sm.OLS(spy, pca_factors[['PC 1', 'PC 2', 'PC 3']]).fit()
print(pcr.summary())

                                 OLS Regression Results                                
Dep. Variable:                    SPY   R-squared (uncentered):                   0.926
Model:                            OLS   Adj. R-squared (uncentered):              0.925
Method:                 Least Squares   F-statistic:                              1808.
Date:                Fri, 23 Aug 2024   Prob (F-statistic):                   1.66e-245
Time:                        12:06:19   Log-Likelihood:                          1584.9
No. Observations:                 439   AIC:                                     -3164.
Df Residuals:                     436   BIC:                                     -3152.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

2. Calculate the Partial Least Squares estimation of SPY on the 22 single-name equities. Model it for 3 factors. Report the r-squared.



In [47]:
X_PLS = returns
y_PLS = spy

model_PLS = PLSRegression(n_components=3).fit(X_PLS, y_PLS)

print('PLS R-squared: ' + str(round(model_PLS.score(X_PLS, y_PLS),3)))

PLS R-squared: 0.983



3. Compare the results between these two approaches and against penalized regression seen in the past homework.

PCR and PLS both seek to maximize the ability to explain the variation in y variable, and therefore they will have high $R^2$ in-sample. When using LASSO or Ridge as our model, we are conservatively forming factors, and penalizing for additional factors. This makes in-sample $R^2$ lower as we saw in Homework #5, but may make more robust OOS predictions.

***